In [9]:
import pandas as pd
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [10]:
#from google.colab import drive
#drive.mount('/content/drive')

In [11]:
df = pd.read_csv("/usr/Train_TP2_Datos_2020-2C.csv") 
df = df[df["Stage"] != "Qualification"]
df = df[df["Stage"] != "Negotiation"]
df = df[df["Stage"] != "Proposal"]

In [12]:
#creating labelEncoder
le = preprocessing.LabelEncoder()
# Converting string labels into numbers.
#territory_encoded = le.fit_transform(df["Territory"])
#billing_country_encoded = le.fit_transform(df["Billing_Country"])
#op_type_encoded = le.fit_transform(df["Opportunity_Type"])
#prod_type_encoded = le.fit_transform(df["Product_Type"])
#label = le.fit_transform(df["Stage"])

for feature in df.columns:
  df[feature] = le.fit_transform(df[feature])

In [13]:
df.head()

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,Source,Billing_Country,Account_Name,Opportunity_Name,Opportunity_ID,Sales_Contract_No,Account_Owner,Opportunity_Owner,Account_Type,Opportunity_Type,Quote_Type,Delivery_Terms,Opportunity_Created_Date,Brand,Product_Type,Size,Product_Category_B,Price,Currency,Last_Activity,Quote_Expiry_Date,Last_Modified_Date,Last_Modified_By,Product_Family,Product_Name,ASP_Currency,ASP,ASP_(converted)_Currency,ASP_(converted),Planned_Delivery_Start_Date,Planned_Delivery_End_Date,Month,Delivery_Quarter,Delivery_Year,Actual_Delivery_Date,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Stage,Prod_Category_A
0,16094,2,46,1,1,1,1,0,4,536,0,52,1295,2166,0,5266,33,7,2,1,1,2,375,24,0,0,0,51,1,0,62,456,6,200,445,1,200,0,368,578,733,7,1,0,0,10,1,7954,1,6281,0,0
1,16093,2,46,0,0,0,0,0,4,536,0,52,1295,2169,1,3100,33,10,2,1,1,2,375,24,0,0,0,51,1,0,32,10,9,200,3,1,203,0,375,31,37,0,0,0,0,0,1,2123,1,1008,1,0
2,15839,1,43,0,0,0,0,0,4,415,12,77,694,5309,2,5008,43,52,4,1,1,4,380,24,0,0,0,51,1,0,975,634,52,205,437,4,184,0,289,53,51,0,0,0,0,0,4,2887,4,1728,1,0
3,7418,1,43,1,0,1,0,0,5,640,4,77,181,4879,3,5266,46,52,4,11,1,1,380,25,2,6,10,39,2,0,995,320,52,107,323,4,203,0,329,338,490,28,0,2,0,14,4,8137,4,6426,0,0
4,7415,1,43,1,0,1,0,0,5,640,4,77,181,8532,4,5266,46,52,4,11,1,1,380,25,2,6,10,39,2,0,702,320,52,107,323,4,203,0,329,338,399,28,0,2,0,25,4,8431,4,6658,0,0


In [14]:
#features=list(zip(territory_encoded,billing_country_encoded,op_type_encoded,prod_type_encoded))
features = df.values

In [23]:
X_train, X_test, y_train, y_test = train_test_split(features, df["Stage"], test_size=0.3) # 70% training and 30% tes

log_loss_min = 100
mejor_accuracy = 0
mejor_i = 0
for i in range(1, 100):
  print(i)
  model = KNeighborsClassifier(n_neighbors=i)
  model.fit(X_train,y_train)
  y_pred = model.predict(X_test)
  log_loss_actual = metrics.log_loss(y_test, y_pred)
  if (log_loss_actual < log_loss_min):
        log_loss_min = log_loss_actual
        mejor_accuracy = metrics.accuracy_score(y_test, y_pred)
        mejor_i = i


print(mejor_i, log_loss_min, mejor_accuracy)
    

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
1 2.1616808797623057 0.9374136229022705


In [26]:
model2 = KNeighborsClassifier(n_neighbors=1)
model2.fit(X_train,y_train)
y2_pred = model2.predict(X_test)
metrics.log_loss(y_test, y2_pred)

2.1616808797623057